In [20]:
%matplotlib inline
import astropy.io.fits as pyfits
import numpy as np
from astropy.utils.data import get_pkg_data_filename
import subprocess as sp
import scipy.ndimage as sci_nd
import glob
import matplotlib.pyplot as mpl
from astropy.table import Table
import sys
import os
from astropy.io import ascii


In [39]:
#from utils on Bruno's github

def select_object_map(xc,yc,segmap,pixscale,radius):
    r"""Filters the segmentation mask of the object by setting all regions
    outside the given radius (in arcseconds) to 0 in the output segmentation
    mask.
    Parameters
    ----------
    xc : float
        the horizontal coordinate (in pixel) of the object center
    yc : float
        the vertical coordinate (in pixel) of the object center
    segmap: int, array
        A binary image array flagging all pixels to be considered for the
        computation. It uses all pixels with non-zero values.
    pixscale : float
        The scale to convert between pixel and arcseconds (given in arcsec/pix).
        If one wants to compute everything in pixel coordinates simply set the
        pixscale to one.
    radius : float
        The distance to consider above which all detected objects are discarded.
        Note that it is only required that a region has at least one pixel at a
        distance smaller than radius to be included in the filtered segmentation
        mask.
    Returns
    -------
    new_map : int, array
        A binary array with all regions containing at least one pixel at a
        distance smaller than the given radius.
    References
    ----------
    Examples
    --------
    """
    Dmat,d=distance_matrix(xc,yc,segmap)
    s_values = np.unique(segmap[Dmat<np.sqrt(2)*radius/pixscale])
    new_map = segmap.copy()
    for s in s_values:
        if s==0:
            continue
        new_map[new_map==s]=-1
    new_map[new_map>0]=0
    return -new_map

def select_object_map_connected(xc,yc,image,segmap,pixscale,radius=0.5):
    r"""Selects a single connected region on the segmentation mask that contains
    the brightest pixel inside the given radius (in arcseconds).
    Parameters
    ----------
    xc : float
        the horizontal coordinate (in pixel) of the object center
    yc : float
        the vertical coordinate (in pixel) of the object center
    img : float, array
        The image array containing the data for which to select the region.
    segmap: int, array
        A binary image array flagging all pixels to be considered for the
        computation. It uses all pixels with non-zero values.
    pixscale : float
        The scale to convert between pixel and arcseconds (given in arcsec/pix).
        If one wants to compute everything in pixel coordinates simply set the
        pixscale to one.
    radius : float
        The distance to consider above which all detected objects are discarded.
        Note that it is only required that a region has at least one pixel at a
        distance smaller than radius to be included in the filtered segmentation
        mask.
    Returns
    -------
    new_map : int, array
        A binary array with the connected region containing the brightest pixel
        at a distance smaller than the given radius.
    References
    ----------
    Examples
    --------
    """
    Regions,Nregions = sci_nd.label(segmap)
#    A= np.array([np.size(Regions[Regions==n]) for n in range(1,Nregions+1)])
    try:
        central_region = image.copy()
        Dmat,d=distance_matrix(xc,yc,segmap)
        central_region[Dmat>(radius/pixscale)]=0.0
        central_region*=segmap
        fmax = np.where(central_region == np.amax(central_region))
        n = Regions[fmax]
##        F= np.array([np.amax(image[Regions==n]) for n in range(1,Nregions+1)])
##        n = np.argmax(F)+1
        Regions[Regions!=n]=0
        Regions[Regions>0]=1

    except ValueError as err:
        ## IN CASE OF NON-DETECTIONS returns empty segmentation map
        print(err)
        pass

    # import matplotlib.pyplot as mpl
    # nFig,nAx = mpl.subplots(1,3)
    # nAx[0].imshow(segmap,cmap='rainbow')
    # nAx[0].plot([xc],[yc],'wx',markersize=20,mew=2)
    # nAx[1].imshow(central_region,cmap='rainbow')
    # nAx[1].plot([xc],[yc],'wx',markersize=20,mew=2)
    # nAx[2].imshow(Regions,cmap='YlGnBu_r')
    # nAx[2].plot([xc],[yc],'rx',markersize=20,mew=2)
    # mpl.show()
    return Regions


def get_center_coords(imgname,ra,dec,hsize=1,verify_limits=True):
    r""" Computes the x,y coordinates of a ra,dec position in a given image.
    Parameters
    ----------
    imgname : string
        The name of the FITS image to use as reference frame
    ra : float
        the right ascension coordinate
    dec: float
        the declination coordinate
    hsize : int, optional
        if given, checks if the computed coordinates are at a distance greater
        than hsize from the image border.
    verify_limits : bool, optional
        if True, verifies the distance of the coordinates to the image border.
    Returns
    -------
    (xc,yc) : tuple, float
        A tuple with the x and y pixel-coordinates corresponing to the given
        sky coordinates using the input image as reference frame.
    References
    ----------
    Examples
    --------
    """
    hdr=pyfits.getheader(imgname)

    return get_center_coords_hdr(hdr,ra,dec,hsize=1,verify_limits=verify_limits)

def compute_ellipse_distmat(img,xc,yc,q=1.00,ang=0.00,overSampling = 1):
    r"""Compute a matrix with dimensions of the image where in each pixel we
    have the distance to the center xc,yc.
    Parameters
    ----------
    img : float, array
        The image array containing the data for which to compute the distance
        matrix.
    xc : float
        the horizontal coordinate (in pixel) of the ellipse center
    yc : float
        the vertical coordinate (in pixel) of the ellipse center
    q : float, optional
        the axis ratio of the ellipse. If not given, computes the distance
        assuming a circular geometry.
    ang : float, optional
        the position angle (in degrees) of the ellipse. If not given, an angle
        of zero degrees is assumed.
    Returns
    -------
    dmat : float, array
        A 2D array of the same shape as the input image where each pixel encodes
        its distance to the input center coordinates and the given geometry.
    References
    ----------
    Examples
    --------
    """
    ang_rad = np.radians(ang)
    X,Y = np.meshgrid(np.arange(img.shape[0]*overSampling),np.arange(img.shape[1]*overSampling))
    rX=(X-xc*overSampling)/overSampling*np.cos(ang_rad)-(Y-yc*overSampling)/overSampling*np.sin(ang_rad)
    rY=(X-xc*overSampling)/overSampling*np.sin(ang_rad)+(Y-yc*overSampling)/overSampling*np.cos(ang_rad)
    dmat = np.sqrt(rX*rX+(1/(q*q))*rY*rY)
    return dmat

In [2]:
#from galfit on Bruno's github


def get_fixpars_default():
    r""" Returns the default dictionary containing the information on whether
    or not to fix any parameter of the fit. By default, all parameters are
    not fixed.
    Parameters
    ----------
    Returns
    -------
    fixpars : dict
        A dictionary for each of the sersic parameters setting the fix/free key.
    References
    ----------
    Examples
    --------
    """
    return {'x':1,'y':1,'m':1,'re':1,'n':1,'q':1,'pa':1,'sky':1}



def write_object(model,x,y,m,re,n,ba,pa,num,fixpars=None):
    r""" Returns a string object containing a general description for a galaxy
    profile model in GALFIT, with the input parameters as first guesses.
    Parameters
    ----------
    model : str
    Returns
    -------
    References
    ----------
    Examples
    --------
    """
    if fixpars is None:
        fixpars=get_fixpars_default()

    objString = ""
    objString += "#Object number: %i\n"%(num)
    objString += " 0) %s             # Object type\n"%(model)
    objString += " 1) %6.4f %6.4f  %i %i    # position x, y        [pixel]\n"%(x,y,fixpars['x'],fixpars['y'])
    objString += " 3) %4.4f      %i       # total magnitude\n"%(m,fixpars['m'])
    objString += " 4) %4.4f       %i       #     R_e              [Pixels]\n"%(re,fixpars['re'])
    objString += " 5) %4.4f       %i       # Sersic exponent (deVauc=4, expdisk=1)\n"%(n,fixpars['n'])
    objString += " 9) %4.4f       %i       # axis ratio (b/a)   \n"%(ba,fixpars['q'])
    objString += "10) %4.4f       %i       # position angle (PA)  [Degrees: Up=0, Left=90]\n"%(pa,fixpars['pa'])
    objString += " Z) 0                  #  Skip this model in output image?  (yes=1, no=0)\n"
    objString += " \n"
    return objString


def input_file(f,modelsString,magzpt,sky,x_range,y_range,sconvbox,pixscale,imgname='input.fits',outname="output.fits",psfname='none',maskname="none",signame='none',fixpars=None):
    r""" Writes an input galfit file with the provided parameters.
    Parameters
    ----------
    f : file pointer
        Python file pointer to be written
    modelsString : str
        A single string containing the starting parameters of the models to be
        used. Model strings can be generated by the write_object function.
    magzpt : float
        Magnitude zeropoint of the image data. Needed for correct model
        magnitudes.
    sky : float
        Initial value for sky background (assumed to be constant)
    x_range : tuple, int
        X-axis boundaries to be used in image fitting (useful to fit a subset
        of a larger image).
    y_range : tuple, int
        Same as z_range, but for y-axis boundaries.
    sconvbox : int
        Size of the convolution box to be applied to the model image (should be
        at least the size of the PSF image).
    pixscale : float
        Pixel scale (in arcsecond/pixel) of the input image data
    imgname : str, optional
        Name of the image with the input data (default - input.fits)
    outname : str, optional
        Name of the output image block to be created by GALFIT
        (default - output.fits)
    psfname : str, optional
        Name of the psf fits file, to be used for model convolution
        (default - none, no psf used)
    maskname : str, optional
        Name of the mask file to be used in the fit. It mask bad pixel values
        and others (e.g. neighbor objects) that are ignored by the fitting
        algorithm (default - none, no mask used).
    signame : str, optional
        Error image corresponding to the input data.
        (default - none, sigma image created internally).
    fixpars : dict
        Dictionary which controls if parameters are fixed or not for the fit.
        This is used here to control wether the sky background is fixed or a
        free parameter for the fit.
    Returns
    -------
        None
        It writes to the file "f" and returns nothing.
    References
    ----------
    Examples
    --------
    """
    if fixpars is None:
        fixpars=get_fixpars_default()

    assert len(x_range)==len(y_range)==2,"x_range,y_range must have two elements"
    assert x_range[1]>x_range[0],"x_range must be sorted in ascendent order"
    assert y_range[1]>y_range[0],"y_range must be sorted in ascendent order"

    f.write("================================================================================\n")
    f.write("# IMAGE and GALFIT CONTROL PARAMETERS\n")
    f.write("A) %s         # Input data image (FITS file)\n"%imgname)
    f.write("B) %s        # Output data image block\n"%outname)
    f.write("C) %s                # Sigma image name (made from data if blank or 'none' \n"%signame)
    f.write("D) %s         # Input PSF image and (optional) diffusion kernel\n"%psfname)
    f.write("E) 1                   # PSF fine sampling factor relative to data \n")
    f.write("F) %s                # Bad pixel mask (FITS image or ASCII coord list)\n"%maskname)
    f.write("G) none                # File with parameter constraints (ASCII file) \n")
    f.write("H) %i    %i   %i    %i # Image region to fit (xmin xmax ymin ymax)\n"%(x_range[0],x_range[1],y_range[0],y_range[1]))
    f.write("I) %i    %i          # Size of the convolution box (x y)\n"%(sconvbox,sconvbox))
    f.write("J) %7.5f             # Magnitude photometric zeropoint \n"%magzpt)
    f.write("K) %.3f %.3f        # Plate scale (dx dy)   [arcsec per pixel]\n"%(pixscale,pixscale))
    f.write("O) regular             # Display type (regular, curses, both)\n")
    f.write("P) 0                   # Options: 0=normal run; 1,2=make model/imgblock and quit\n")
    f.write("\n")
    f.write("# INITIAL FITTING PARAMETERS\n")
    f.write("#\n")
    f.write("#For object type, the allowed functions are:\n")
    f.write("#nuker, sersic, expdisk, devauc, king, psf, gaussian, moffat,\n")
    f.write("#ferrer, and sky.\n")
    f.write("#\n")
    f.write("#Hidden parameters will only appear when theyre specified:\n")
    f.write("#C0 (diskyness/boxyness),\n")
    f.write("#Fn (n=integer, Azimuthal Fourier Modes).\n")
    f.write("#R0-R10 (PA rotation, for creating spiral structures).\n")
    f.write("#\n")
    f.write("# ------------------------------------------------------------------------------\n")
    f.write("#  par)    par value(s)    fit toggle(s)   parameter description\n")
    f.write("# ------------------------------------------------------------------------------\n")
    f.write("\n")

    f.write(modelsString)

    f.write("# Object: Sky\n")
    f.write(" 0) sky                    #  object type\n")
    f.write(" 1) %7.4f      %i          #  sky background at center of fitting region [ADUs]\n"%(sky,fixpars['sky']))
    f.write(" 2) 0.0000      0          #  dsky/dx (sky gradient in x)\n")
    f.write(" 3) 0.0000      0          #  dsky/dy (sky gradient in y)\n")
    f.write(" Z) 0                      #  output option (0 = resid., 1 = Dont subtract)")
    f.close()
    return


In [ ]:
#{filter}_check.fits files are the segmentation maps

#creating mask maps 

#select_object_map_connected(xc,yc,image,segmap,pixscale,radius=0.5)

In [7]:
#generating initial parameter guesses

#fixpars = get_fixpars_default()
#fixpars

{'x': 1, 'y': 1, 'm': 1, 're': 1, 'n': 1, 'q': 1, 'pa': 1, 'sky': 1}

# Starting to make parameters needed for GalFits input config file
## Creating Parameter -- Mask_Name 

In [35]:
#One Filter to start
filters = {"F105W"}

#Main Idea: Mask_name = segmentation_map - mask_map
#                       segmentation_maps = {filter}_check.fits files 


#creating mask_map for one selected object first [before doing all objects in a cycle]
#first object = value #528 [large circle to upper left of image,& SE of obvious star there]
detected_objs_file = ascii.read('testPSF_F105W.cat')

##exploring file to pull values for selected object
print('PLEASE NOTE!!!!! OBJECT_INDEX = DETECTED_OBJECT_VALUE - 1')
#detected_objs_file #full table with column headings
#detected_objs_file["X_IMAGE"] #calls column as expected
#detected_objs_file.info() #headers, basically commented out top of file
#detected_objs_file[0] #first detected object value = index#
detected_objs_file[527:529] #object i selected to focus on first
#detected_objs_file[528]["X_IMAGE"] #should be 2271.2468 and it is!
#detected_objs_file[528]["Y_IMAGE"] #3737.6321


PLEASE NOTE!! OBJECT_INDEX = DETECTED_OBJECT_VALUE - 1


MAG_AUTO,KRON_RADIUS,ISOAREA_IMAGE,XPEAK_IMAGE,YPEAK_IMAGE,X_IMAGE,Y_IMAGE,A_IMAGE,THETA_IMAGE,MU_MAX,ELLIPTICITY,CLASS_STAR,FLUX_RADIUS
mag,,pix2,pix,pix,pix,pix,pix,deg,mag / arcsec2,,,pix
float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
20.1915,5.61,730,2398,3808,2399.4817,3808.0471,7.293,-35.82,21.6271,0.165,0.67,13.46
24.1104,8.75,8,2271,3738,2271.2468,3737.6321,0.973,-17.85,23.8714,0.326,0.35,3.633


In [42]:
detected_objs_file.info()

<Table length=570>
     name      dtype       unit                            description                        
------------- ------- ------------- ----------------------------------------------------------
     MAG_AUTO float64           mag                    Kron-like elliptical aperture magnitude
  KRON_RADIUS float64               Kron apertures in units of A or B                         
ISOAREA_IMAGE   int64          pix2                    Isophotal area above Analysis threshold
  XPEAK_IMAGE   int64           pix                        x-coordinate of the brightest pixel
  YPEAK_IMAGE   int64           pix                        y-coordinate of the brightest pixel
      X_IMAGE float64           pix                                    Object position along x
      Y_IMAGE float64           pix                                    Object position along y
      A_IMAGE float64           pix                               Profile RMS along major axis
  THETA_IMAGE float64          

In [81]:
#defing input parameters for fcn 

xc = detected_objs_file[527]["X_IMAGE"] #in pix
yc = detected_objs_file[527]["Y_IMAGE"] #in pix
image = pyfits.open('/data1/rowland/elgordo_F105W.fits')
segmap_file = pyfits.open('F105W_check.fits')
segmap = segmap_file[0].data
pixscale = 0.06
#radius=0.5 ##DEFAULT value for reference 
select_object_map_connected(xc,yc,image,segmap,pixscale)


ValueError: Big-endian buffer not supported on little-endian compiler

In [84]:
segmap_file.info()
segmap_file.close()

Filename: F105W_check.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   (4538, 4824)   int32   


In [80]:
#regions, nregions = sci_nd.label(segmap)
#segmap needs to be an array
''' 
for i in segmap[0].data:
    #print(len(i))
    for val in i:
        if val > 0:
            print(val)
    else:
        pass
'''
#np.shape(segmap[0].data) #(4824, 4538)
#segmap.info()
#segmap[0].data


#converting to pandas df to fix big-endian buffer - little-endian complier error via google solution
#dat = Table.read('F105W_check.fits', format='fits')
#Pdb = dat.to_pandas()
from astropy.io import fits
import pandas
with fits.open('F105W_check.fits') as data:
    #data.byteswap().newbyteorder()
    Pdb = pandas.DataFrame(data[0].data)
Pdb.field('fieldname').dtype
#Pdb = 
#(Pdb)temps.dtype
#dtype('>f8')
#Pdb.temps.astype('f8').dtype
#dtype('float64')
#(Pdb)

AttributeError: 'DataFrame' object has no attribute 'field'

## All Inputs ready to make GalFit input file

In [ ]:
#once all parameters are made can use this to generate an input file for galfit
pixscale = 0.06

#input_file(f,modelsString,magzpt,sky,x_range,y_range,sconvbox,pixscale,imgname='input.fits',outname="output.fits",psfname='none',maskname="none",signame='none',fixpars=None)